<a href="https://colab.research.google.com/github/Oksana0020/DL-with-PyTorch/blob/main/Fully_connected_layers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Adding fully connected layers and model summary

Step 1: Setting Up PyTorch Environment

In [1]:

import torch
import torch.nn as nn
import torch.nn.functional as F


Step 2: Defining Fully Connected Layers in PyTorch

In [2]:
class CNNWithFC(nn.Module):
    def __init__(self, input_channels=1, image_height=28, image_width=28, num_classes=10):
        super(CNNWithFC, self).__init__()

        self.input_channels = input_channels
        self.image_height = image_height
        self.image_width = image_width
        self.num_classes = num_classes

        # --------------------------
        # Convolutional layers
        # Conv2d(in_channels, out_channels, kernel_size, padding)
        # With kernel_size=3 and padding=1, spatial dims (H, W) are preserved.
        # Output shape after conv1: (batch_size, 32, image_height, image_width)
        # --------------------------
        self.conv1 = nn.Conv2d(self.input_channels, 32, kernel_size=3, padding=1)

        # Output shape after pool1: (batch_size, 32, image_height/2, image_width/2)
        # After conv2 (padding=1): (batch_size, 64, image_height/2, image_width/2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)

        # --------------------------
        # Max pooling layer
        # Each pooling operation halves the spatial dimensions
        # --------------------------
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # --------------------------
        # Calculate flattened size after conv + pool layers
        # After conv1 (dims preserved) then pool1 (dims halved):
        # height_after_pool1 = image_height // 2
        # width_after_pool1  = image_width  // 2
        #
        # After conv2 (dims preserved) then pool2 (dims halved again):
        pooled_height = self.image_height // 2 // 2
        pooled_width  = self.image_width  // 2 // 2

        # The number of output channels from conv2 is 64
        self.fc1_in_features = 64 * pooled_height * pooled_width

        # --------------------------
        # Fully connected layers
        # --------------------------
        self.fc1 = nn.Linear(self.fc1_in_features, 128)
        self.fc2 = nn.Linear(128, self.num_classes)  # output layer (logits)

    def forward(self, x):
        # Convolutional Block 1
        x = self.conv1(x)
        x = F.relu(x)       # ReLU activation
        x = self.pool(x)    # Max pooling

        # Convolutional Block 2
        x = self.conv2(x)
        x = F.relu(x)       # ReLU activation
        x = self.pool(x)    # Max pooling

        # Flatten before FC layers
        x = x.view(x.size(0), -1)

        # Fully Connected Block
        x = F.relu(self.fc1(x))
        x = self.fc2(x)     # raw logits (no softmax for CrossEntropyLoss)
        return x


Step 3: Visualizing the Model Summary

In [3]:
# Ensure torchsummary is installed.
# If not installed in your environment, uncomment the next line:
# !pip install torchsummary

from torchsummary import summary

# Define input parameters (MNIST example)
input_channels = 1
image_height = 28
image_width = 28
num_classes = 10

# Create model instance
model = CNNWithFC(
    input_channels=input_channels,
    image_height=image_height,
    image_width=image_width,
    num_classes=num_classes
)

# Print model summary
# input_size format: (channels, height, width)
summary(model, input_size=(input_channels, image_height, image_width))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
         MaxPool2d-2           [-1, 32, 14, 14]               0
            Conv2d-3           [-1, 64, 14, 14]          18,496
         MaxPool2d-4             [-1, 64, 7, 7]               0
            Linear-5                  [-1, 128]         401,536
            Linear-6                   [-1, 10]           1,290
Total params: 421,642
Trainable params: 421,642
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.36
Params size (MB): 1.61
Estimated Total Size (MB): 1.97
----------------------------------------------------------------
